In [1]:
pip install -U transpose-data

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
  Preparing metadata (setup.py) ... done
  Created wheel for transpose-data: filename=transpose_data-2.0.4-py3-none-any.whl size=46428 sha256=ba851026063a58289f1b824e00c4d335f7ab286eccd277216996a55da4424e70
  Stored in directory: /home/stubbs/.cache/pip/wheels/42/ed/74/ffc5161ae8011f3fa2ed35094e50457dc875df1959cc3e258a
Successfully built transpose-data
  Attempting uninstall: transpose-data
    Found existing installation: transpose-data 2.0.3
    Uninstalling transpose-data-2.0.3:
      Successfully uninstalled transpose-data-2.0.3
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import requests
from transpose import Transpose
import networkx as nx
from pyvis.network import Network

# Collect all data on the treasury

In [5]:
testing_wallet = str.lower("0x6d956A6Aaca9BB7A0e4D34b6924729F856c641dE")
multisig_1 = str.lower("0x722f531740937fc21A2FaC7648670C7f2872A1c7")
multisig_2 = str.lower("0xDAAf6D0ab259053Bb601eeE69880F30C7d8e5853")
multisig_3 = str.lower("0x3C3ca4E5AbF0C1Bec701375ff31342d90D8C435E")

contract_address_usdc = str.lower("0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48")
contract_address_dai = str.lower('0x6B175474E89094C44Da98b954EedeAC495271d0F')
contract_address_frax = str.lower('0x853d955aCEf822Db058eb8505911ED77F175b99e')
contract_address_usdt = str.lower('0xdAC17F958D2ee523a2206206994597C13D831ec7')
contract_address_rook = str.lower('0xfA5047c9c78B8877af97BDcb85Db743fD7313d4a')

contract_address_weth = str.lower('0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2')

In [6]:
ALCHEMY_API_KEY = "zryBeieKVRvOjPoKo-o01twywIo5pbsB"
TRANSPOSE_API_KEY = "jPPHbOLtJ0oHWqsXkkzxHt53pA1cCX3v"
FLIPSIDE_CRYPTO_API = "02dcaa9a-040f-49e5-921b-fd5e2f33e370"
api=Transpose(TRANSPOSE_API_KEY)

In [7]:
url = f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"

payload = {
    "id": 1,
    "jsonrpc": "2.0",
    "method": "alchemy_getTokenBalances",
    "params": [[str(contract_address_usdc), str(contract_address_dai), str(contract_address_frax), str(contract_address_usdt)]]
}
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}
response = requests.post(url, json=payload, headers=headers)
print(response.text)

{"jsonrpc":"2.0","id":1,"error":{"code":-32602,"message":"invalid 1st argument: address address was not string"}}


# Use the Transpose Python SDK to get the current token balances

In [8]:
wallet_balances = pd.json_normalize(api.token.tokens_by_owner(testing_wallet, limit=500).to_dict())[['contract_address', 'name', 'symbol', 'decimals', 'balance']]
token_metadata = pd.json_normalize(api.token.tokens_by_contract_address([contract_address_usdc, contract_address_dai, contract_address_frax, contract_address_usdt, contract_address_weth, contract_address_rook]).to_dict())[['contract_address', 'name', 'symbol', 'decimals']] 
# print(wallet_balances)

Get metadata for tokens

In [9]:
def token_amount(df):
    return df['balance']/pow(10, df['decimals'])

scam_coins = wallet_balances[wallet_balances['contract_address'] == '0x1f068a896560632a4d2E05044BD7F03834f1A465'].index
bal = wallet_balances.drop(scam_coins).assign(amount = lambda x: token_amount(x))
bal = bal.assign(amount = lambda x: token_amount(x))
bal.head()
                                                

,contract_address,name,symbol,decimals,balance,amount
1,0x6B175474E89094C44Da98b954EedeAC495271d0F,Dai Stablecoin,DAI,18,114612052262274105369285,114612.052262
2,0x853d955aCEf822Db058eb8505911ED77F175b99e,Frax,FRAX,18,30225111549633591272,30.225112
3,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USD Coin,USDC,6,40245484075,40245.484075
4,0xdAC17F958D2ee523a2206206994597C13D831ec7,Tether USD,USDT,6,444384208106,444384.208106



### Token Transfers


In [10]:
def token_amount(df):
    return df['quantity']/pow(10,df['decimals'])

token_transfers_all = pd.json_normalize(api.token.transfers_by_account(testing_wallet, limit=500).to_dict()).merge(token_metadata, how='left', left_on='contract_address', right_on='contract_address').assign(token_amount = lambda x: token_amount(x))
# token_transfers_all_n_metadata = token_transfers_all.merge(token_metadata, how='left', left_on = 'contract_address', right_on = 'contract_address')
# token_transfers_all_n_metadata = token_transfers_all_n_metadata.assign(token_amount = lambda x: token_amount(x))
token_transfers_all.head()

,activity_id,contract_address,block_number,log_index,transaction_hash,timestamp,category,operator,from,to,quantity,name,symbol,decimals,token_amount
0,15398691000424000000,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,15398691,424,0x9700adc049848fc6cd8ab54fa253b373ff1075cce8f0...,2022-08-23T20:17:56Z,send,None,0xDAAf6D0ab259053Bb601eeE69880F30C7d8e5853,0x6d956A6Aaca9BB7A0e4D34b6924729F856c641dE,1000000000,USD Coin,USDC,6.0,1000.0
1,15398919000193000000,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,15398919,193,0xe1d0ef2bf790a8d6d189445f1ece91a23159f68d1bc3...,2022-08-23T21:06:08Z,send,None,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3,0x6d956A6Aaca9BB7A0e4D34b6924729F856c641dE,588234842084259509,Wrapped Ether,WETH,18.0,0.588235
2,15398919000194000000,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,15398919,194,0xe1d0ef2bf790a8d6d189445f1ece91a23159f68d1bc3...,2022-08-23T21:06:08Z,send,None,0x6d956A6Aaca9BB7A0e4D34b6924729F856c641dE,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3,999999231,USD Coin,USDC,6.0,999.999231
3,15403482000002000000,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,15403482,2,0xe6622cb02252a6b6b8d4d5e1a36fc9dc6e414a9112e5...,2022-08-24T14:39:28Z,send,None,0x6d956A6Aaca9BB7A0e4D34b6924729F856c641dE,0xDAAf6D0ab259053Bb601eeE69880F30C7d8e5853,588234842084259509,Wrapped Ether,WETH,18.0,0.588235
4,15403634000054000000,0x6B175474E89094C44Da98b954EedeAC495271d0F,15403634,54,0x0bf3603109a80ecc94535cb22fd3965e5b36b0339aed...,2022-08-24T15:16:58Z,send,None,0x9a67F1940164d0318612b497E8e6038f902a00a4,0x6d956A6Aaca9BB7A0e4D34b6924729F856c641dE,100000000000000000000,Dai Stablecoin,DAI,18.0,100.0


In [11]:
names_dict = {'0x6d956A6Aaca9BB7A0e4D34b6924729F856c641dE':'Testing Wallet'}
token_transfers = token_transfers_all[['transaction_hash', 'timestamp', 'contract_address','symbol', 'token_amount', 'from', 'to']].rename(columns={'transaction_hash':'tx_hash', 'contract_address':'token_contract_address', 'from':'sender', 'to':'receiver'}).replace({'sender': names_dict, 'receiver':names_dict})
# names_dict = {'0x6d956A6Aaca9BB7A0e4D34b6924729F856c641dE':'Testing Wallet'}
# token_transfers.replace({'sender': names_dict, 'receiver':names_dict}, inplace=True)
token_transfers.head(25)

,tx_hash,timestamp,token_contract_address,symbol,token_amount,sender,receiver
0,0x9700adc049848fc6cd8ab54fa253b373ff1075cce8f0...,2022-08-23T20:17:56Z,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,1000.0,0xDAAf6D0ab259053Bb601eeE69880F30C7d8e5853,Testing Wallet
1,0xe1d0ef2bf790a8d6d189445f1ece91a23159f68d1bc3...,2022-08-23T21:06:08Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,WETH,0.588235,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3,Testing Wallet
2,0xe1d0ef2bf790a8d6d189445f1ece91a23159f68d1bc3...,2022-08-23T21:06:08Z,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,999.999231,Testing Wallet,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3
3,0xe6622cb02252a6b6b8d4d5e1a36fc9dc6e414a9112e5...,2022-08-24T14:39:28Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,WETH,0.588235,Testing Wallet,0xDAAf6D0ab259053Bb601eeE69880F30C7d8e5853
4,0x0bf3603109a80ecc94535cb22fd3965e5b36b0339aed...,2022-08-24T15:16:58Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,100.0,0x9a67F1940164d0318612b497E8e6038f902a00a4,Testing Wallet
5,0xb1a107695e2361c6526bf9acaf75f9f1c830cbeb5b07...,2022-08-24T15:19:46Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,599900.0,0x9a67F1940164d0318612b497E8e6038f902a00a4,Testing Wallet
6,0xf35b16dd00e4b46c7d2fc459dd73e606bd2d0ce7bb3f...,2022-08-24T15:49:52Z,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,149974.80346,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3,Testing Wallet
7,0xf35b16dd00e4b46c7d2fc459dd73e606bd2d0ce7bb3f...,2022-08-24T15:49:52Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,149999.793426,Testing Wallet,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3
8,0x0ec0ab6d601b1bef441212be011f3212cd52f30ccbe8...,2022-08-24T16:45:16Z,0xdAC17F958D2ee523a2206206994597C13D831ec7,USDT,149974.915534,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3,Testing Wallet
9,0x0ec0ab6d601b1bef441212be011f3212cd52f30ccbe8...,2022-08-24T16:45:16Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,149999.905518,Testing Wallet,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3


In [12]:
token_transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 0 to 38
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   tx_hash                 39 non-null     object
 1   timestamp               39 non-null     object
 2   token_contract_address  39 non-null     object
 3   symbol                  38 non-null     object
 4   token_amount            38 non-null     object
 5   sender                  39 non-null     object
 6   receiver                39 non-null     object
dtypes: object(7)
memory usage: 2.4+ KB


In [13]:
token_transfer_from = token_transfers[token_transfers['sender']=='Testing Wallet']
token_transfer_to = token_transfers[token_transfers['receiver']=='Testing Wallet']
token_sent = token_transfer_from.merge(token_transfer_to, how='left', left_on=['tx_hash'], right_on=['tx_hash'])[['tx_hash', 'timestamp_x', 'token_contract_address_x', 'symbol_x', 'token_amount_x', 'sender_x', 'receiver_x', 'token_contract_address_y', 'symbol_y', 'token_amount_y']].dropna(subset=['token_contract_address_y'])

# token_sent = token_transfer_from.merge(token_transfer_to, how='outer', left_on=['tx_hash'], right_on=['tx_hash'])[['tx_hash', 'timestamp_x', 'token_contract_address_x', 'symbol_x', 'token_amount_x', 'sent_from_x', 'token_contract_address_y', 'symbol_y', 'token_amount_y', 'sent_from_y']].sort_values('tx_hash')
# token_sent = token_transfer_from.merge(token_transfer_to, how='left', left_on=['tx_hash'], right_on=['tx_hash'])[['tx_hash', 'timestamp_x', 'token_contract_address_x', 'symbol_x', 'token_amount_x', 'sent_from_x', 'token_contract_address_y', 'symbol_y', 'token_amount_y', 'sent_from_y']].sort_values('tx_hash')
# token_sent.dropna(subset=['token_contract_address_y'], inplace=True)
# token_sent = token_swap.rename(columns={'timestamp_x':'timestamp', 'token_contract_address_x':'token_contract_x', 'sent_from_x':'sent_from', 'token_contract_address_y': 'token_contract_y', 'sent_from_y': 'sent_to'})
token_sent.head()
# [['tx_hash', 'timestamp_x', 'token_symbol_x', 'token_amount_x', 'to_contract

,tx_hash,timestamp_x,token_contract_address_x,symbol_x,token_amount_x,sender_x,receiver_x,token_contract_address_y,symbol_y,token_amount_y
0,0xe1d0ef2bf790a8d6d189445f1ece91a23159f68d1bc3...,2022-08-23T21:06:08Z,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,999.999231,Testing Wallet,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,WETH,0.588235
2,0xf35b16dd00e4b46c7d2fc459dd73e606bd2d0ce7bb3f...,2022-08-24T15:49:52Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,149999.793426,Testing Wallet,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,149974.80346
3,0x0ec0ab6d601b1bef441212be011f3212cd52f30ccbe8...,2022-08-24T16:45:16Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,149999.905518,Testing Wallet,0x5492A302991751fd6b5c051d85Be48Acc9fEC4D3,0xdAC17F958D2ee523a2206206994597C13D831ec7,USDT,149974.915534
4,0xf47086364a47a8c16f52861b425a731d5f7bb3813d38...,2022-08-24T16:45:16Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,148925.98394,Testing Wallet,0xd8c07491cAA1eDF960db3Ceff387426d53942ea0,0x853d955aCEf822Db058eb8505911ED77F175b99e,FRAX,148901.172871
5,0x9dfe02883f0f5d3315032a50f1a8dc8e6b941aff1575...,2022-08-24T16:54:49Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,150197.893403,Testing Wallet,0xd8c07491cAA1eDF960db3Ceff387426d53942ea0,0x853d955aCEf822Db058eb8505911ED77F175b99e,FRAX,150197.893403


In [14]:
def create_trade_path(df):
    return f"{df['symbol_x']}-->{df['symbol_y']}"

token_sent['path'] = token_sent.apply(create_trade_path, axis=1)

In [15]:
token_sent[['path','tx_hash']].groupby('path').count()

,tx_hash
path,
DAI-->FRAX,3
DAI-->USDC,2
DAI-->USDT,2
FRAX-->DAI,1
FRAX-->USDC,1
FRAX-->USDT,1
USDC-->DAI,2
USDC-->FRAX,1
USDC-->USDT,1


In [16]:
token_sent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 17
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   tx_hash                   17 non-null     object
 1   timestamp_x               17 non-null     object
 2   token_contract_address_x  17 non-null     object
 3   symbol_x                  17 non-null     object
 4   token_amount_x            17 non-null     object
 5   sender_x                  17 non-null     object
 6   receiver_x                17 non-null     object
 7   token_contract_address_y  17 non-null     object
 8   symbol_y                  17 non-null     object
 9   token_amount_y            17 non-null     object
 10  path                      17 non-null     object
dtypes: object(11)
memory usage: 1.6+ KB


In [17]:
token_swap.iloc[0]['tx_hash']

NameError: name 'token_swap' is not defined

In [18]:
df_network = token_sent[['symbol_x', 'symbol_y', 'token_amount_x']]
df_network

,symbol_x,symbol_y,token_amount_x
0,USDC,WETH,999.999231
2,DAI,USDC,149999.793426
3,DAI,USDT,149999.905518
4,DAI,FRAX,148925.98394
5,DAI,FRAX,150197.893403
6,USDC,FRAX,149946.923948
7,USDT,FRAX,149973.951935
8,DAI,FRAX,876.248074
9,FRAX,USDT,199999.751482
10,FRAX,USDC,199999.830329


In [19]:
import matplotlib.pyplot as plt
import networkx as nx

In [20]:
nodes = ['USDC', 'USDT', 'FRAX', 'DAI']
df_network=df_network[(df_network['symbol_x'].isin(nodes)) & (df_network['symbol_y'].isin(nodes))].dropna()

df_network

,symbol_x,symbol_y,token_amount_x
2,DAI,USDC,149999.793426
3,DAI,USDT,149999.905518
4,DAI,FRAX,148925.98394
5,DAI,FRAX,150197.893403
6,USDC,FRAX,149946.923948
7,USDT,FRAX,149973.951935
8,DAI,FRAX,876.248074
9,FRAX,USDT,199999.751482
10,FRAX,USDC,199999.830329
11,USDC,USDT,149946.304574


In [21]:
trades=df_network.groupby(['symbol_x', 'symbol_y'])['token_amount_x'].agg(['sum', 'count']).reset_index().sort_values('sum', ascending=False)
trades

,symbol_x,symbol_y,sum,count
0,DAI,FRAX,300000.125417,3
2,DAI,USDT,248827.419278,2
4,FRAX,USDC,199999.830329,1
5,FRAX,USDT,199999.751482,1
3,FRAX,DAI,199866.383308,1
1,DAI,USDC,165644.668468,2
9,USDT,FRAX,149973.951935,1
7,USDC,FRAX,149946.923948,1
8,USDC,USDT,149946.304574,1
6,USDC,DAI,29432.38346,2


In [51]:
G = nx.from_pandas_edgelist(trades, 'symbol_x', 'symbol_y', 'sum',create_using=nx.DiGraph())
# G=nx.DiGraph(trades[['symbol_x','symbol_y']])

In [52]:
G.degree()

DiDegreeView({'DAI': 5, 'FRAX': 6, 'USDT': 5, 'USDC': 6})

In [99]:
DG=nx.DiGraph()

# trade_dict = trades.to_dict('records')
# edge_value_list = []
# for row in trade_dict:
#     node_1 = row['symbol_x']
#     node_2 = row['symbol_y']
#     amount = row['sum']
#     count = row['count']
#     DG.add_nodes_from()
#     edge_value_list.append((node_1, node_2, amount))
# print(edge_value_list)
                           
# DG.add_weighted_edges_from(edge_value_list)


trade_dict = trades.to_dict('records')
edge_value_list = []
node_list = []
amount_list = []
count_list = []

for row in trade_dict:
    node_1 = row['symbol_x']
    node_2 = row['symbol_y']
    DG.add_edge(node_1, node_2)
    amount = row['sum']
    count = row['count']
    edge_value_list.append((node_1, node_2, amount))
    
    node_list.append(node_1)
    amount_list.append(amount)
    count_list.append(count)
    # DG.add_edge(node_1, node_2, weight=amount)
# DG.add_nodes(node_list, value = amount_list)
# print(edge_value_list)
                           
# DG.add_weighted_edges_from(edge_value_list)


In [106]:
pos = nx.spring_layout(DG)
nx.draw_networkx_nodes(DG,pos, node_size = weight)

NameError: name 'weight' is not defined

In [102]:


nt = Network()
nt.from_nx(DG)
nt.prep_notebook()
nt.show('nx.html')

In [131]:
G.adj

AdjacencyView({'DAI': {'FRAX': {'sum': 300000.12541704025, 'count': 3}, 'USDT': {'sum': 248827.4192783559, 'count': 2}, 'USDC': {'sum': 165644.6684676, 'count': 2}}, 'FRAX': {'USDC': {'sum': 199999.8303294545, 'count': 1}, 'USDT': {'sum': 199999.7514818499, 'count': 1}, 'DAI': {'sum': 199866.383308423, 'count': 1}}, 'USDT': {'FRAX': {'sum': 149973.951935, 'count': 1}, 'USDC': {'sum': 4178.550226, 'count': 1}}, 'USDC': {'FRAX': {'sum': 149946.923948, 'count': 1}, 'USDT': {'sum': 149946.304574, 'count': 1}, 'DAI': {'sum': 29432.38346, 'count': 2}}})

In [133]:
for u, v, sum in G.edges(data='sum'):
    if weight is not None:
        print(u, v, weight)
        pass

In [137]:
for n, nbrsdict in G.adjacency():
    for nbr, eattr in nbrsdict.items():
        if 'sum' in eattr:
            print(nbr, eattr)

FRAX {'sum': 300000.12541704025, 'count': 3}
USDT {'sum': 248827.4192783559, 'count': 2}
USDC {'sum': 165644.6684676, 'count': 2}
USDC {'sum': 199999.8303294545, 'count': 1}
USDT {'sum': 199999.7514818499, 'count': 1}
DAI {'sum': 199866.383308423, 'count': 1}
FRAX {'sum': 149973.951935, 'count': 1}
USDC {'sum': 4178.550226, 'count': 1}
FRAX {'sum': 149946.923948, 'count': 1}
USDT {'sum': 149946.304574, 'count': 1}
DAI {'sum': 29432.38346, 'count': 2}
